# Creating an Isochrone Map of High Schools and Transit Stops Within Walking Distance
## By: Caleb Huang

In [1]:
#importing libraries for coding
import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import osmnx as ox
import contextily as ctx
import folium
from shapely.geometry import Point
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import re

In [2]:
# adding Muni Stops and checking on an example
munistops = gpd.read_file('data/Muni Stops_20250119.zip')
munistops.sample(1)

objectid                        stopname trapezesto rucusstopa  stopid  \
1269   73605.0  Fillmore St&Sutter St SE-NS/BZ   FILLSUT1   FILLSUTT  4640.0   

       latitude   longitude  accessibil   atstreet     onstreet  ... shelter  \
1269  37.786051 -122.433244         NaN  SUTTER ST  FILLMORE ST  ...     0.0   

         insert_tim      sde_id  signupid supervisor  date_data_  \
1269  1729767266000  15730304.0     147.0          5  2024-11-02   

        time_data_ date_dat_2    time_dat_2                     geometry  
1269  20:47:21.000 2024-11-04  09:42:39.000  POINT (-122.43324 37.78605)  

[1 rows x 23 columns]

In [3]:
# looking at the different columns
munistops.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3285 entries, 0 to 3284
Data columns (total 23 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   objectid    3285 non-null   float64       
 1   stopname    3285 non-null   object        
 2   trapezesto  3285 non-null   object        
 3   rucusstopa  2930 non-null   object        
 4   stopid      3285 non-null   float64       
 5   latitude    3285 non-null   float64       
 6   longitude   3285 non-null   float64       
 7   accessibil  657 non-null    float64       
 8   atstreet    3194 non-null   object        
 9   onstreet    3274 non-null   object        
 10  position    2922 non-null   object        
 11  orientatio  2944 non-null   object        
 12  servicepla  1646 non-null   object        
 13  shelter     3285 non-null   float64       
 14  insert_tim  3285 non-null   object        
 15  sde_id      3285 non-null   float64       
 16  signupid    3285

In [4]:
# filtering out the columns I need
munistops=munistops[['stopname', 'latitude', 'longitude', 'geometry']]
munistops.sample()

stopname   latitude   longitude  \
268  Diamond St&24TH St SE-NS/PS  37.751075 -122.436219   

                        geometry  
268  POINT (-122.43622 37.75108)

In [5]:
# importing schools file
schools = gpd.read_file('data/public_hs.csv')
schools

Campus Name CCSF Entity Grade Range  \
0               Thurgood Marshall High School       SFUSD        9-12   
1         Phillip And Sala Burton High School       SFUSD        9-12   
2               George Washington High School       SFUSD        9-12   
3                 Abraham Lincoln High School       SFUSD        9-12   
4   Gateway High School / Kipp SF Bay Academy       SFUSD        9-12   
5                         Galileo High School       SFUSD        9-12   
6                          Balboa High School       SFUSD        9-12   
7        City Arts And Leadership High School       SFUSD        9-12   
8                    Independence High School       SFUSD        9-12   
9                Raoul Wallenberg High School       SFUSD        9-12   
10                       Downtown High School       SFUSD        9-12   
11    San Francisco International High School       SFUSD        9-12   
12                    June Jordan High School       SFUSD        9-12   
13                 John O'Connell High School       SFUSD        9-12   
14                        Mission High School       SFUSD        9-12   
15                   Ida B. Wells High School       SFUSD        9-12   
16       Life Learning Academy Charter School       SFUSD        9-12   

                  Location 1  
0   (37.735676, -122.401888)  
1   (37.720628, -122.405299)  
2   (37.777851, -122.492040)  
3   (37.747810, -122.481411)  
4   (37.783140, -122.437727)  
5    (37.81309, -122.423980)  
6   (37.721418, -122.440282)  
7   (37.721384, -122.445052)  
8   (37.763329, -122.464052)  
9   (37.780102, -122.446239)  
10  (37.761380, -122.404130)  
11  (37.761742, -122.401229)  
12  (37.719829, -122.425494)  
13  (37.759519, -122.414725)  
14  (37.761442, -122.426946)  
15  (37.775253, -122.434362)  
16  (37.825512, -122.367996)

Here, I am creating the walking maps of each individual school of study. I am defining a function and creating a for loop to create a map for each school.

In [6]:
# splitting the location column into separate lat and lon columns
schools['latitude'] = schools['Location 1'].apply(lambda x: float(x.strip('()').split(',')[0]))
schools['longitude'] = schools['Location 1'].apply(lambda x: float(x.strip('()').split(',')[1]))

# Create Point objects from the extracted latitude and longitude values
schools['geometry'] = schools.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)

# Assign the Point objects to a new geometry column in the GeoDataFrame and set the CRS to WGS 84
schools = gpd.GeoDataFrame(schools, geometry='geometry', crs='EPSG:4326')

# Ensure munistops has a CRS set before converting it to the CRS of schools
if munistops.crs is None:
    munistops.set_crs('EPSG:4326', inplace=True)  # Assuming the original CRS is WGS 84

# Convert munistops to the same CRS as schools
munistops = munistops.to_crs(schools.crs)

# Define a function for creating the map
def schools_walking_map(school, place, munistops):
    latlon = school.geometry.coords[0][::-1]  # Reverse the order to get (latitude, longitude)
    network_type = 'walk' 
    trip_times = [5, 10, 15] 
    meters_per_minute = 75 

    # Manually set the CRS to epsg:4326 for the graph
    G = ox.graph_from_point(latlon, network_type=network_type, dist=2000, simplify=True, custom_filter=None)
    G = ox.project_graph(G, to_crs='epsg:3857')
    gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)

    minx, miny, maxx, maxy = gdf_nodes.geometry.total_bounds
    centroid_x = (maxx - minx) / 2 + minx
    centroid_y = (maxy - miny) / 2 + miny

    center_node = ox.distance.nearest_nodes(G, Y=centroid_y, X=centroid_x)

    gdf_edges['walk_time'] = gdf_edges['length'] / meters_per_minute

    cmap = 'RdYlGn'
    iso_colors = ox.plot.get_colors(n=len(trip_times), cmap=cmap, start=0)

    time_color = list(zip(trip_times, iso_colors))
    time_color.reverse()

    for time, color in list(time_color):
        # for each trip time, create an egograph of nodes that fall within that distance
        subgraph = nx.ego_graph(G, center_node, radius=time, distance='walk_time')

        # for each of those nodes, update the gdf_nodes dataframe and assign it with its associated distance color
        for node in subgraph.nodes():
            gdf_nodes.loc[node, 'time'] = time
            gdf_nodes.loc[node, 'color'] = color

    gdf_nodes['color'] = gdf_nodes['color'].fillna('#cccccc')

    isochrones = gdf_nodes.dissolve(by="time")
    isochrones = isochrones.convex_hull.reset_index(name='geometry')
    isochrones.sort_values(by='time', ascending=False, inplace=True)
    iso_colors = ['#f7fcb9', '#addd8e', '#31a354']

    fig, ax = plt.subplots(figsize=(10, 15))

    # add the isochrone boundary
    isochrones.boundary.plot(
        ax=ax,
        alpha=1,
        linestyle='--',
        color=iso_colors,
        lw=2
    )

    # add the isochrones
    isochrones.plot(
        ax=ax,
        alpha=0.2,
        categorical=True,  # even though it is numeric, treat it as categorical
        color=iso_colors,
    )

    # add the center node in red
    gdf_nodes.loc[[center_node]].plot(
        ax=ax,
        color='r',
        marker='x',
        markersize=50
    )

    # Debug: Print bounds and centroid to understand the issue
    print (latlon)
    print(f"Bounds: {gdf_nodes.total_bounds}")
    print(f"Centroid: ({centroid_x}, {centroid_y})")

    # Ensure CRS of munistops matches CRS of isochrones before spatial join
    munistops = munistops.to_crs(isochrones.crs)

    # Debug: Print CRS to ensure they match
    print(f"munistops CRS: {munistops.crs}")
    print(f"isochrones CRS: {isochrones.crs}")

    # Add transit stops within the isochrone area
    transit_stops_within_isochrone = gpd.sjoin(munistops, isochrones, predicate='intersects')
    
    # Debug: Print the transit stops within isochrone to understand the issue
    print(f"Transit stops within isochrone: {transit_stops_within_isochrone}")

    if not transit_stops_within_isochrone.empty:
        transit_stops_within_isochrone.plot(
            ax=ax,
            color='blue',
            markersize=10,
            label='Transit Stop'
        )

    # Set the aspect ratio manually to avoid ValueError
    ax.set_aspect('equal')

    # build custom legend
    legend_elements = [
        # add the center node to the legend
        Line2D([0], [0], marker='x', color='red', linestyle='', label='Start location', markersize=6),
        # add the transit stops within isochrone to the legend
        Line2D([0], [0], marker='o', color='blue', linestyle='', label='Transit Stop', markersize=6)
    ]

    # sort back to small time to large time
    # time_color.sort(reverse=False)
    time_color.reverse()

    # loop through the list of time/colors and add each to the legend
    for time, color in list(time_color):
        legend_item = Patch(facecolor=color, edgecolor=color, linestyle='--', linewidth=1, label=str(time) + ' minutes', alpha=0.4)
        legend_elements.append(legend_item)

    # add the legend
    ax.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(1, 1))  

    # add a title
    ax.set_title(f'How far are transit stops from {place}?', fontsize=15, pad=20)

    # hide the axis
    ax.axis('off')

    # add the basemap
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)

    # Sanitize the place string to create a valid filename
    sanitized_place = re.sub(r'[^a-zA-Z0-9]', '_', place)

    # Save the figure
    plt.savefig(f'{sanitized_place}_walking_map.png')
    plt.close()

# Iterate over each unique location and create a map
for idx, school in schools.iterrows():
    place = school['Campus Name']  # Replace with the actual column name for the school name
    schools_walking_map(school, place, munistops)

(37.735676, -122.401888)
Bounds: [-13628247.23838399   4539622.31083674 -13623184.28322737
   4544684.10503905]
Centroid: (-13625715.760805683, 4542153.207937896)
munistops CRS: epsg:3857
isochrones CRS: epsg:3857
Transit stops within isochrone:                                 stopname   latitude   longitude  \
40        Oakdale Ave&Toland St NE-NS/PS  37.740966 -122.401416   
161     Industrial St&Elmira St SE-FS/SB  37.738790 -122.403730   
205     Silver Ave&San Bruno Ave S-NS/BZ  37.732410 -122.405672   
205     Silver Ave&San Bruno Ave S-NS/BZ  37.732410 -122.405672   
225        Silver Ave&Revere Ave W-FS/PS  37.736361 -122.398888   
...                                  ...        ...         ...   
3038  Silver Ave&Charter Oak Ave N-NS/BZ  37.733275 -122.404297   
3075  San Bruno Ave&Thornton Ave E-NS/BZ  37.730260 -122.404620   
3180   Industrial St&Bayshore Blvd SE-FS  37.737943 -122.406780   
3209             Revere Ave&Lane St W-NS  37.731690 -122.390290   
3265      Newhall

## Counting Transit Stops
To understand the differences between schools and their access to transit, I wanted to create a count of the transit stops within a 15 minute walk of the school. I will create a function to count the stops and iterate it through each isochorone map. After this, I will create a dataframe and then use that dataframe to create bar charts. 

In [19]:
import pandas as pd
schools['latitude'] = schools['Location 1'].apply(lambda x: float(x.strip('()').split(',')[0]))
schools['longitude'] = schools['Location 1'].apply(lambda x: float(x.strip('()').split(',')[1]))

schools['geometry'] = schools.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)

schools = gpd.GeoDataFrame(schools, geometry='geometry', crs='EPSG:4326')

if munistops.crs is None:
    munistops.set_crs('EPSG:4326', inplace=True)  

# Convert munistops to the same CRS as schools
munistops = munistops.to_crs(schools.crs)

# Function to count transit stops within isochrone zones
def count_transit_stops(school, munistops):
    latlon = school.geometry.coords[0][::-1]  # Reverse the order to get (latitude, longitude)
    network_type = 'walk' 
    trip_times = [5, 10, 15] 
    meters_per_minute = 75 

    G = ox.graph_from_point(latlon, network_type=network_type, dist=2000, simplify=True, custom_filter=None)
    G = ox.project_graph(G, to_crs='epsg:3857')
    gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)

    minx, miny, maxx, maxy = gdf_nodes.geometry.total_bounds
    centroid_x = (maxx - minx) / 2 + minx
    centroid_y = (maxy - miny) / 2 + miny

    center_node = ox.distance.nearest_nodes(G, Y=centroid_y, X=centroid_x)

    gdf_edges['walk_time'] = gdf_edges['length'] / meters_per_minute

    cmap = 'RdYlGn'
    iso_colors = ox.plot.get_colors(n=len(trip_times), cmap=cmap, start=0)

    time_color = list(zip(trip_times, iso_colors))
    time_color.reverse()

    for time, color in list(time_color):
        # for each trip time, create an egograph of nodes that fall within that distance
        subgraph = nx.ego_graph(G, center_node, radius=time, distance='walk_time')
        
        for node in subgraph.nodes():
            gdf_nodes.loc[node, 'time'] = time
            gdf_nodes.loc[node, 'color'] = color

    gdf_nodes['color'] = gdf_nodes['color'].fillna('#cccccc')

    isochrones = gdf_nodes.dissolve(by="time")
    isochrones = isochrones.convex_hull.reset_index(name='geometry')
    isochrones.sort_values(by='time', ascending=False, inplace=True)

    munistops = munistops.to_crs(isochrones.crs)

    # Add transit stops within the isochrone area
    transit_stops_within_isochrone = gpd.sjoin(munistops, isochrones, predicate='intersects')

    # Count the number of transit stops within the isochrone
    num_transit_stops = transit_stops_within_isochrone.shape[0]

    return num_transit_stops

results = []

# Iterate over each unique location and count the number of transit stops within the isochrone zones
for idx, school in schools.iterrows():
    place = school['Campus Name']  # Replace with the actual column name for the school name
    num_transit_stops = count_transit_stops(school, munistops)
    results.append({'School': place, 'Transit Stops': num_transit_stops})

results_df = pd.DataFrame(results)
print(results_df)

                                       School  Transit Stops
0               Thurgood Marshall High School            116
1         Phillip And Sala Burton High School            190
2               George Washington High School             98
3                 Abraham Lincoln High School            193
4   Gateway High School / Kipp SF Bay Academy            148
5                         Galileo High School            151
6                          Balboa High School             47
7        City Arts And Leadership High School             80
8                    Independence High School             82
9                Raoul Wallenberg High School             57
10                       Downtown High School            165
11    San Francisco International High School            184
12                    June Jordan High School            112
13                 John O'Connell High School             12
14                        Mission High School             48
15                   Ida

In [20]:
results_df.drop([16],axis=0,inplace=True)

In [21]:
transit_stops_list=results_df.sort_values(by='Transit Stops',ascending = True)
transit_stops_list

School  Transit Stops
13                 John O'Connell High School             12
15                   Ida B. Wells High School             35
6                          Balboa High School             47
14                        Mission High School             48
9                Raoul Wallenberg High School             57
7        City Arts And Leadership High School             80
8                    Independence High School             82
2               George Washington High School             98
12                    June Jordan High School            112
0               Thurgood Marshall High School            116
4   Gateway High School / Kipp SF Bay Academy            148
5                         Galileo High School            151
10                       Downtown High School            165
11    San Francisco International High School            184
1         Phillip And Sala Burton High School            190
3                 Abraham Lincoln High School            193

In [22]:
plt.figure(figsize=(12, 8))
plt.bar(transit_stops_list['School'], transit_stops_list['Transit Stops'], color='steelblue')
plt.xlabel('School')
plt.ylabel('Number of Transit Stops')
plt.title('Number of Transit Stops within a 15 Minute Walk for Each School')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('schools_transit_stops.png')
plt.show()

In [23]:
least_stops=transit_stops_list.head(5)
least_stops

School  Transit Stops
13    John O'Connell High School             12
15      Ida B. Wells High School             35
6             Balboa High School             47
14           Mission High School             48
9   Raoul Wallenberg High School             57

In [24]:
plt.figure(figsize=(12, 8))
plt.bar(least_stops['School'], least_stops['Transit Stops'], color='steelblue')
plt.xlabel('Schools With the Least Number of Transit Stops')
plt.ylabel('Number of Transit Stops')
plt.title('Number of Transit Stops within a 15 Minute Walk for Each School')
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig('top5_least_stops.png')
plt.show()

In [25]:
most_stops=transit_stops_list.tail(5)
most_stops

School  Transit Stops
5                       Galileo High School            151
10                     Downtown High School            165
11  San Francisco International High School            184
1       Phillip And Sala Burton High School            190
3               Abraham Lincoln High School            193

In [26]:
plt.figure(figsize=(12, 8))
plt.bar(most_stops['School'], most_stops['Transit Stops'], color='steelblue')
plt.xlabel('Schools With the Most Number of Transit Stops')
plt.ylabel('Number of Transit Stops')
plt.title('Number of Transit Stops within a 15 Minute Walk for Each School')
plt.xticks(rotation=35)
plt.tight_layout()
plt.savefig('top5_most_stops.png')
plt.show()